In [2]:
from google.colab import drive
drive.mount('/content/grdive')

Drive already mounted at /content/grdive; to attempt to forcibly remount, call drive.mount("/content/grdive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import math 
from collections import deque
from numpy import array
import random
import datetime
import calendar
import time

In [0]:
df =pd.read_excel('/content/grdive/My Drive/CFD Machine Learning/CFD.xlsx')

In [0]:
df=df.iloc[:,3:9]

In [6]:
df.head()

,ccx,ccy,ccz,U1,U2,U3
0,-15.66665,-15.66665,0.333333,1.000053,0.000269,0.000074
1,-15.00000,-15.66665,0.333333,1.000582,0.000954,0.000260
2,-14.33335,-15.66665,0.333333,1.001449,0.001724,0.000427
3,-13.66665,-15.66665,0.333333,1.002579,0.002579,0.000593
4,-13.00000,-15.66665,0.333333,1.003905,0.003499,0.000760


In [0]:
df['distance']=np.sqrt((df['ccx']**2)+(df['ccy']**2)+(df['ccz']**2))

In [8]:
df.head()

,ccx,ccy,ccz,U1,U2,U3,distance
0,-15.66665,-15.66665,0.333333,1.000053,0.000269,0.000074,22.158496
1,-15.00000,-15.66665,0.333333,1.000582,0.000954,0.000260,21.692281
2,-14.33335,-15.66665,0.333333,1.001449,0.001724,0.000427,21.236760
3,-13.66665,-15.66665,0.333333,1.002579,0.002579,0.000593,20.792603
4,-13.00000,-15.66665,0.333333,1.003905,0.003499,0.000760,20.360625


In [0]:
df=df.sort_values('distance')

In [10]:
df.head()

,ccx,ccy,ccz,U1,U2,U3,distance
276101,-3.549367,0.873805,0.165214,0.157078,0.312564,0.086993,3.659076
276075,-3.549369,-0.873805,0.165215,0.186573,0.425258,0.081138,3.659077
276008,-0.873900,-3.549809,0.165215,1.250538,0.322960,0.072068,3.659527
276166,-0.873901,3.549815,0.165213,1.157566,0.301015,0.068508,3.659534
2717,0.873998,3.550097,0.165215,0.683874,0.066385,0.019641,3.659830


In [0]:
X=df.iloc[:,:].values

In [12]:
X.shape

(346167, 7)

In [0]:
sequential_data=[]
nearby_velocities =deque(maxlen=101)

In [0]:
#Creates the complete sequential data to be split into train and test sequences
for i in range(0,len(X)-1):
  nearby_velocities.append([n for n in X[i]])
  if len(nearby_velocities) ==101:
    sequential_data.append([np.array(nearby_velocities),np.array(nearby_velocities[50][3:6])])

    

In [15]:
len(sequential_data)

346066

In [16]:
df.iloc[50]

ccx        -1.178359
ccy        -3.529415
ccz         0.165999
U1          1.258596
U2          0.435250
U3          0.065031
distance    3.724628
Name: 276007, dtype: float64

In [17]:
sequential_data

Output hidden; open in https://colab.research.google.com to view.

In [0]:
random.shuffle(sequential_data)

In [19]:
#AFTER SHUFFLING
sequential_data

Output hidden; open in https://colab.research.google.com to view.

In [0]:
X_train=array([n[0] for n in sequential_data[0:50000]])
X_validate=array([n[0] for n in sequential_data[50000:55000]])

In [21]:
print(X_train.shape)
print(X_validate.shape)

(50000, 101, 7)
(5000, 101, 7)


In [0]:
#X_train=np.delete(X_train,50,1) #The first number 50 is index value and 1 means 2nd dimension
#X_validate=np.delete(X_validate,50,1)

In [23]:
print(X_train.shape)
print(X_validate.shape)

(50000, 101, 7)
(5000, 101, 7)


In [0]:
Y_train=array([n[1] for n in sequential_data[0:50000]])
Y_validate=array([n[1] for n in sequential_data[50000:55000]])  #Y_test will be used for validation

In [25]:
print(Y_train.shape)
print(Y_validate.shape)

(50000, 3)
(5000, 3)


In [0]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
sc_Y=StandardScaler()

In [0]:
X_train[:,:,0]=sc_X.fit_transform(X_train[:,:,0])
X_train[:,:,1]=sc_X.fit_transform(X_train[:,:,1])
X_train[:,:,2]=sc_X.fit_transform(X_train[:,:,2])
X_train[:,:,3]=sc_X.fit_transform(X_train[:,:,3])
X_train[:,:,4]=sc_X.fit_transform(X_train[:,:,4])
X_train[:,:,5]=sc_X.fit_transform(X_train[:,:,5])
X_train[:,:,6]=sc_X.fit_transform(X_train[:,:,6])

In [0]:
X_validate[:,:,0]=sc_X.fit_transform(X_validate[:,:,0])
X_validate[:,:,1]=sc_X.fit_transform(X_validate[:,:,1])
X_validate[:,:,2]=sc_X.fit_transform(X_validate[:,:,2])
X_validate[:,:,3]=sc_X.fit_transform(X_validate[:,:,3])
X_validate[:,:,4]=sc_X.fit_transform(X_validate[:,:,4])
X_validate[:,:,5]=sc_X.fit_transform(X_validate[:,:,5])
X_validate[:,:,6]=sc_X.fit_transform(X_validate[:,:,6])

In [0]:
Y_train=sc_Y.fit_transform(Y_train)
Y_validate=sc_Y.fit_transform(Y_validate)

In [0]:
                                    #RECURSIVE DEEP NEURAL NETWORK

import torch
import torch.nn as nn
import torchvision.datasets as datasets  #import the dataset
import torchvision.transforms as transforms  #
from torch.autograd import Variable  #autograd is a part of pytorch 0.3 but can also be used with pytorch 0.4
from torch.utils.data import Dataset
import sys

In [0]:
                                    #CONVERT NUMPY TO PYTORCH 
X_train=torch.tensor(X_train)
Y_train=torch.tensor(Y_train)
X_validate=torch.tensor(X_validate)
Y_validate=torch.tensor(Y_validate)

#CONVERT PYTORCH TENSOR TO VARIABLE FORMAT
X_train=Variable(X_train)
Y_train=Variable(Y_train)
X_validate=Variable(X_validate)
Y_validate=Variable(Y_validate)


In [32]:
print(X_train.shape)
print(X_validate.shape)
print(Y_train.shape)
print(Y_validate.shape)

torch.Size([50000, 101, 7])
torch.Size([5000, 101, 7])
torch.Size([50000, 3])
torch.Size([5000, 3])


In [0]:
#WHEN WE ARE USING OUR OWN DATASET THEN WE NEED TO CREATE A DATASET CLASS WHICH INHERITS FROM PREDEFINED 
#DATASET CLASS OF torch.utils.data library
class Dataset(Dataset):
  def __init__(self,x,y):
    self.x=x
    self.y=y
  
  def __getitem__(self,index):
    return self.x[index], self.y[index]
  
  def __len__(self):
    return len(self.x)

In [0]:
train_dataset=Dataset(X_train,Y_train)
validate_dataset=Dataset(X_validate,Y_validate)

In [35]:
print(train_dataset.x.shape)
print(train_dataset.y.shape)
print(validate_dataset.x.shape)
print(validate_dataset.y.shape)

torch.Size([50000, 101, 7])
torch.Size([50000, 3])
torch.Size([5000, 101, 7])
torch.Size([5000, 3])


In [0]:
#MAKE DATA ITERABLE BY LOADING IT TO A LOADER. Shuffe the data to make it independent of order
#Batch size is the number of elements in a batch
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=200,
                                        shuffle=True)

In [0]:
#MAKE DATA ITERABLE BY LOADING IT TO A LOADER. Shuffe the data to make it independent of order
#Batch size is the number of elements in a batch
validation_loader=torch.utils.data.DataLoader(dataset=validate_dataset,
                                        batch_size=200,
                                        shuffle=False)

In [38]:
len(validation_loader)

25

In [39]:
print(train_dataset.x.shape)
print(train_dataset.y.shape)
print(validate_dataset.x.shape)
print(validate_dataset.y.shape)

torch.Size([50000, 101, 7])
torch.Size([50000, 3])
torch.Size([5000, 101, 7])
torch.Size([5000, 3])


In [40]:
#THIS IS ONLY FOR DISPLAY PURPOSE TO UNDERSTAND THE DATA
print("There is {} batches in the dataset".format(len(train_loader)))
shown=0
for (x,y) in train_loader:
  if shown==1:
    break
  print("For one batch there is:")
  print("Data: {}, Type: {} ".format(x.shape,x.dtype))
  print("Data: {}, Type: {} ".format(y.shape,y.dtype))
  shown+=1
  

There is 250 batches in the dataset
For one batch there is:
Data: torch.Size([200, 101, 7]), Type: torch.float64 
Data: torch.Size([200, 3]), Type: torch.float64 


In [0]:
#Input of neural net 2 nodes and output of the neural net is 3 nodes
# Neural Net using 3 hidden layers each containing 3 nodes 

input_size=7
hidden_size=3

class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.bi_rcnn1=torch.nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=False, bidirectional=True)   
    self.fc1=torch.nn.Linear(6,3)
              
    
    #Input hidden hidden output  =>layer format
  def forward(self,x):
      out,hidden=self.bi_rcnn1(x)  
      out=out[:,50]  #Middle value-1 6 input values to fully connected layer
      out=self.fc1(out)
      return out  
  
  def init_hidden(self):
      # Initialize hidden and cell states
      # (num_layers * num_directions, batch, hidden_size)
      return Variable(torch.zeros(1, 200, 3))

    

In [0]:
# Instantiate LSTM model
model = Model()

In [43]:
print(model.parameters)

<bound method Module.parameters of Model(
  (bi_rcnn1): LSTM(7, 3, bidirectional=True)
  (fc1): Linear(in_features=6, out_features=3, bias=True)
)>


In [44]:
criterion=torch.nn.MSELoss(size_average=True)   #Loss Criterion
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [0]:
start_time=calendar.timegm(time.gmtime())

In [0]:
epochs=100   #We have done it for 100 epochs 
for epoch in range(epochs):
  for x, y in train_loader:
    x=Variable(x.float())
    y=Variable(y.float())
    outputs=model(x)
    optimizer.zero_grad()
    
    loss=criterion(outputs,y)
    loss.backward()
    optimizer.step()
  
  print("EPOCH {} COMPLETED".format(epoch))
  print("Training loss: {}".format(loss.item()))

EPOCH 0 COMPLETED
Training loss: 0.015724048018455505
EPOCH 1 COMPLETED
Training loss: 0.004564778879284859
EPOCH 2 COMPLETED
Training loss: 0.0029136554803699255
EPOCH 3 COMPLETED
Training loss: 0.002227662829682231
EPOCH 4 COMPLETED
Training loss: 0.0010888011893257499
EPOCH 5 COMPLETED
Training loss: 0.0008741933270357549
EPOCH 6 COMPLETED
Training loss: 0.0006558470777235925
EPOCH 7 COMPLETED
Training loss: 0.0012235242174938321
EPOCH 8 COMPLETED
Training loss: 0.0009248810820281506
EPOCH 9 COMPLETED
Training loss: 0.0005811198498122394
EPOCH 10 COMPLETED
Training loss: 0.0010360830929130316
EPOCH 11 COMPLETED
Training loss: 0.0005143588641658425
EPOCH 12 COMPLETED
Training loss: 0.0003986163646914065
EPOCH 13 COMPLETED
Training loss: 0.00025720364646986127
EPOCH 14 COMPLETED
Training loss: 0.00023839340428821743
EPOCH 15 COMPLETED
Training loss: 0.00027065700851380825
EPOCH 16 COMPLETED
Training loss: 0.0008457271032966673
EPOCH 17 COMPLETED
Training loss: 0.00024775153724476695
E

In [0]:
end_time=calendar.timegm(time.gmtime())

In [0]:
total_time=(end_time-start_time)/3600
print("Time taken to run in hrs is: "+str(total_time))

In [0]:
count=1
for x, y in validation_loader:
    x=Variable(x.float())
    y=Variable(y.float())
    outputs=model(x)
    
    for i,j in zip(outputs,y):
      print("Predicted "+str(i)+" Actual "+str(j))
    print("Batch: {} completed".format(count))
    count=count+1

In [0]:
outputs=sc_Y.inverse_transform(outputs.detach().numpy())
y=sc_Y.inverse_transform(y.detach().numpy())

In [0]:
#VELOCITY IN ACTUAL FORMAT AFTER INVERSE TRANSFORMATION FOR THE LAST BATCH ie for 200 observations
for i,j in zip(outputs,y):
  print("Predicted "+str(i)+" Actual "+str(j))

In [0]:
#R-SQUARED METRIC
R_residual=((outputs[:,1]-y[:,1])**2).sum()
R_total=((outputs[:,1]-np.average(y[:,1]))**2).sum()
R_squared=1-(R_residual/R_total)
print(R_squared)


In [0]:
calendar.timegm(time.gmtime())